In [1]:
import os
from pathlib import Path

import numpy as np
import xarray as xr

In [2]:
dirpath = os.path.join(Path.home(), "FjordsSim_data", "varna")
ds = xr.open_dataset(os.path.join(dirpath, "Varna_BRY_1lat.nc"))

In [3]:
depth = np.loadtxt(os.path.join(dirpath, "Varna_topo_channels.txt"))

In [4]:
# Define new dimensions
Ny, Nx = depth.shape

In [5]:
lons = np.linspace(27.640, 27.947, Nx)
lats = np.linspace(43.177, 43.214, Ny)

In [6]:
ds = ds.drop_vars(["latitude", "longitude"])

In [7]:
year = 1991
ds = ds.sel(time=slice(f"{year}-01-01", f"{year}-12-31"))

In [8]:
z_faces = -np.arange(0, 21, 1)
z_centers = [(z_faces[i] + z_faces[i + 1]) / 2 for i in range(len(z_faces) - 1)]
ds = ds.interp(depth=z_centers)

In [9]:
Nt = 365
Nz = len(z_centers)
out_shape = (Nt, Nz, Ny, Nx)

In [ ]:
ds

In [11]:
def tile3D(a, nrep):
    return np.repeat(a[:, :, np.newaxis], nrep, axis=2)

In [12]:
Tout = np.full(out_shape, -999, dtype=np.float32)
Tout_lambda = np.full(out_shape, -999, dtype=np.float32)
Sout = np.full(out_shape, -999, dtype=np.float32)
Sout_lambda = np.full(out_shape, -999, dtype=np.float32)
Cout = np.full(out_shape, -999, dtype=np.float32)
Cout_lambda = np.full(out_shape, -999, dtype=np.float32)
# NUTout = np.full(out_shape, -999, dtype=np.float32)
# NUTout_lambda = np.full(out_shape, -999, dtype=np.float32)
# DOMout = np.full(out_shape, -999, dtype=np.float32)
# DOMout_lambda = np.full(out_shape, -999, dtype=np.float32)
# POMout = np.full(out_shape, -999, dtype=np.float32)
# POMout_lambda = np.full(out_shape, -999, dtype=np.float32)
Uout = np.full(out_shape, -999, dtype=np.float32)
Vout = np.full(out_shape, -999, dtype=np.float32)

In [13]:
Tout[:, :, :, -1] = tile3D(ds["thetao"].values, Ny)
Sout[:, :, :, -1] = tile3D(ds["so"].values, Ny)
Uout[:, :, :, -1] = tile3D(ds["uo"].values, Ny)
Vout[:, :, :, -1] = tile3D(ds["vo"].values, Ny)

## Add river and lambdas

In [14]:
# (Nt, Nz, Ny, Nx)
Tout_lambda[:, :, 13, 0] = 1 / 60 / 60 / 1
Tout_lambda[:, :, :, -1] = 1 / 60 / 60 / 24
Sout[:, :, 13, 0] = 0.5
Sout_lambda[:, :, 13, 0] = 1 / 60 / 60 / 1
Sout_lambda[:, :, :, -1] = 1 / 60 / 60 / 24
Cout[:, :, 13, 0] = 10
Cout_lambda[:, :, 13, 0] = 1 / 60 / 60 / 1
Cout_lambda[:, :, :, -1] = 1 / 60 / 60 / 24

In [15]:
# Create an xarray.Dataset
dsout = xr.Dataset(
    {
        "T": (["time", "Nz", "Ny", "Nx"], Tout),
        "T_lambda": (["time", "Nz", "Ny", "Nx"], Tout_lambda),
        "S": (["time", "Nz", "Ny", "Nx"], Sout),
        "S_lambda": (["time", "Nz", "Ny", "Nx"], Sout_lambda),
        "C": (["time", "Nz", "Ny", "Nx"], Cout),
        "C_lambda": (["time", "Nz", "Ny", "Nx"], Cout_lambda),
        "u": (["time", "Nz", "Ny", "Nx"], Uout),
        "v": (["time", "Nz", "Ny", "Nx"], Vout),
    },
    coords={
        "time": ds.time,
        "Nz": z_centers,
        "Ny": lats,
        "Nx": lons,
    },
)

In [ ]:
dsout

In [17]:
mask = np.zeros_like(dsout["T"], dtype=bool)
mask[:, :, [0, -1], :] = True
mask[:, :, :, [0, -1]] = True

In [18]:
dsout["T"] = xr.where(mask, dsout["T"], np.nan)
dsout["T_lambda"] = xr.where(mask, dsout["T_lambda"], np.nan)
dsout["S"] = xr.where(mask, dsout["S"], np.nan)
dsout["S_lambda"] = xr.where(mask, dsout["S_lambda"], np.nan)
dsout["C"] = xr.where(mask, dsout["C"], np.nan)
dsout["C_lambda"] = xr.where(mask, dsout["C_lambda"], np.nan)
dsout["u"] = xr.where(mask, dsout["u"], np.nan)
dsout["v"] = xr.where(mask, dsout["v"], np.nan)

In [ ]:
dsout.T.isel(time=0, Nz=-1).plot(vmin=0)

In [ ]:
dsout.T_lambda.isel(time=0, Nz=-1).plot(vmin=0)

In [ ]:
dsout.S.isel(time=0, Nz=-1).plot(cmap="viridis")

In [ ]:
dsout.S_lambda.isel(time=0, Nz=-1).plot(cmap="viridis")

In [ ]:
dsout.C.isel(time=0, Nz=-1).plot(cmap="viridis")

In [ ]:
dsout.C_lambda.isel(time=0, Nz=-1).plot(cmap="viridis")

In [25]:
dsout = dsout.fillna(-999)

In [26]:
dsout.to_netcdf(os.path.join(dirpath, "Varna_bry.nc"))